# Replication of Krusell-Smith (1998)

##### Caveat: not self-documented perfectly

#### Youngdoo Choi (yoc005@ucsd.edu), March 2023

In [1]:
import numpy as np
from numba import njit

from interpolation import interp
import statsmodels.api as SM
import random

## 0. Setup

In [2]:
# Household parameers
β = 0.99         # discount factor
l_tilde = 0.3271 # labor input

# Firm parameters
α = 0.36  # capital share
δ = 0.025 # depreciation rate

# Employment grids
ϵ_unem = 0.0; ϵ_empl = 1.0; ϵgridsize = 2
ϵgrid = np.array((ϵ_unem, ϵ_empl))

# Capital grids
k_min = 1e-6; k_max = 50.0; kgridsize = 151
curv = 1.5 # curvature
kgrid = np.array([k_min + k_max * (k/(kgridsize-1))**curv for k in range(kgridsize)])

# Aggregate capital grids
K_min = 5.0; K_max = 20.0; Kgridsize = 26
Kgrid = np.linspace(K_min, K_max, Kgridsize)

# TFP grids
z_b = 0.99; z_g = 1.01; zgridsize = 2
zgrid = np.array((z_b, z_g))

# Additional grids
u_b = 0.1; u_g = 0.04
ugrid = np.array((u_b, u_g))
Lgrid = (1-ugrid) * l_tilde

# Etc
tol = 1e-6 # tolerance
max_iter = 10000
update_weight = 0.2 # updating weight of coefficients

In [3]:
# Transition matrix
# Step 1: Calculate each entry
u_b = 0.1  # unemployment rate in bad times
u_g = 0.04 # unemployment rate in good times

π_bb = 1 - 1/8  # average duration of bad times = 8 quarters
π_gg = 1 - 1/8  # average duration of good times = 8 quarters
π_bg = 1 - π_gg # π_bg + π_gg = 1
π_gb = 1 - π_bb # π_gb + π_bb = 1

π_bb00 = 1 - 1/2.5     # average duration of unemployment in bad times
π_gg00 = 1 - 1/1.5     # average duration of unemployment in good times
π_bb01 = π_bb - π_bb00 # π_bb00 + π_bb01 = π_bb
π_gg01 = π_gg - π_gg00 # π_gg00 + π_gg01 = π_gg

π_bg00 = 0.75 * π_gg00 / π_gg * π_bg
π_gb00 = 1.25 * π_bb00 / π_bb * π_gb
π_bg01 = π_bg - π_bg00 # π_bg00 + π_bg01 = π_bg
π_gb01 = π_gb - π_gb00 # π_gb00 + π_gb01 = π_gb

π_bb10 = u_b*(1-π_bb00/π_bb) * π_bb/(1-u_b)     # u_b*π_bb00/π_bb + (1-u_b)*π_bb10/π_bb = u_b
π_gg10 = u_g*(1-π_gg00/π_gg) * π_gg/(1-u_g)     # u_g*π_gg00/π_gg + (1-u_g)*π_gg10/π_gg = u_g
π_bg10 = (u_g - u_b*π_bg00/π_bg) * π_bg/(1-u_b) # u_b*π_bg00/π_bg + (1-u_b)*π_bg10/π_bg = u_g
π_gb10 = (u_b - u_g*π_gb00/π_gb) * π_gb/(1-u_g) # u_g*π_gb00/π_gb + (1-u_g)*π_gb10/π_gb = u_b
π_bb11 = π_bb - π_bb10 # π_bb10 + π_bb11 = π_bb
π_gg11 = π_gg - π_gg10 # π_gg10 + π_gg11 = π_gg
π_bg11 = π_bg - π_bg10 # π_bg10 + π_bg11 = π_bg
π_gb11 = π_gb - π_gb10 # π_gb10 + π_gb11 = π_gb

# Step 2: Compose the transition matrix
Π = np.empty((zgridsize, zgridsize, ϵgridsize, ϵgridsize))
Π[0, 0, 0, 0] = π_bb00; Π[0, 0, 0, 1] = π_bb01; Π[0, 1, 0, 0] = π_bg00; Π[0, 1, 0, 1] = π_bg01
Π[0, 0, 1, 0] = π_bb10; Π[0, 0, 1, 1] = π_bb11; Π[0, 1, 1, 0] = π_bg10; Π[0, 1, 1, 1] = π_bg11
Π[1, 0, 0, 0] = π_gb00; Π[1, 0, 0, 1] = π_gb01; Π[1, 1, 0, 0] = π_gg00; Π[1, 1, 0, 1] = π_gg01
Π[1, 0, 1, 0] = π_gb10; Π[1, 0, 1, 1] = π_gb11; Π[1, 1, 1, 0] = π_gg10; Π[1, 1, 1, 1] = π_gg11

In [4]:
# Simulation parameters
T = 11000  # simulation length
T_0 = 1000 # cut-off length
random.seed(202303) # set seed

# Simulate z series
z_series = np.empty(T)
z0 = z_b; z_series[0] = z0
for i in range(T-1):
    temp = random.random()
    if z_series[i] == zgrid[0]:
        z_series[i+1] = zgrid[0] * float(temp <= π_bb) + zgrid[1] * float(temp > π_bb)
    else:
        z_series[i+1] = zgrid[0] * float(temp <= π_gb) + zgrid[1] * float(temp > π_gb)

## 1. HH Problem

In [5]:
@njit
def EGM_operator(φ_mat, c_old, 
                 zgrid, zgridsize, Kgrid, Kgridsize, ϵgrid, ϵgridsize, kgrid, kgridsize, ugrid, Lgrid, l_tilde, Π, α, δ):
    # An operator to solve HH problem by the endogenous grid method for given coefficients
    
    # Pre-allocation
    c_new = np.empty((zgridsize, Kgridsize, ϵgridsize, kgridsize))
    kp_new = np.empty((zgridsize, Kgridsize, ϵgridsize, kgridsize))
    
    # Fix aggregate states
    for (z_idx, z) in enumerate(zgrid):
        L = (1-ugrid[z_idx]) * l_tilde # today's aggregate labor
        
        for (K_idx, K) in enumerate(Kgrid):
            w = (1-α)*z*(K/L)**α # today's wage
            r = α*z*(K/L)**(α-1) # today's interest rate
            Kp = np.exp(φ_mat[z_idx, 0] + φ_mat[z_idx, 1] * np.log(K)) # tomorrow's aggregate capital
            
            # Fix individual states
            for (ϵ_idx, ϵ) in enumerate(ϵgrid):
                c_tilde = np.empty(kgridsize)
                k_tilde = np.empty(kgridsize)
                
                for (kp_idx, kp) in enumerate(kgrid):
                    c_old_interp1 = lambda a: interp(Kgrid, c_old[0, :, 0, kp_idx], a)
                    c_old_interp2 = lambda a: interp(Kgrid, c_old[1, :, 0, kp_idx], a)
                    c_old_interp3 = lambda a: interp(Kgrid, c_old[0, :, 1, kp_idx], a)
                    c_old_interp4 = lambda a: interp(Kgrid, c_old[1, :, 1, kp_idx], a)
                    
                    Euler_RHS = β * ((1/c_old_interp1(Kp)) * (α*zgrid[0]*(Kp/Lgrid[0])**(α-1) + 1-δ) * Π[z_idx, 0, ϵ_idx, 0] + 
                                     (1/c_old_interp2(Kp)) * (α*zgrid[1]*(Kp/Lgrid[1])**(α-1) + 1-δ) * Π[z_idx, 0, ϵ_idx, 1] + 
                                     (1/c_old_interp3(Kp)) * (α*zgrid[0]*(Kp/Lgrid[0])**(α-1) + 1-δ) * Π[z_idx, 1, ϵ_idx, 0] + 
                                     (1/c_old_interp4(Kp)) * (α*zgrid[1]*(Kp/Lgrid[1])**(α-1) + 1-δ) * Π[z_idx, 1, ϵ_idx, 1])
                    c_tilde[kp_idx] = 1/Euler_RHS
                    k_tilde[kp_idx] = (c_tilde[kp_idx] + kp - w*l_tilde*ϵgrid[ϵ_idx]) / (r+1-δ)
                
                # Check borrowing constraint
                thres_idx = np.where(kgrid > k_tilde[0])[0][0]
                # binding case
                c_new[z_idx, K_idx, ϵ_idx, :thres_idx] = (r+1-δ)*kgrid[:thres_idx] + w*l_tilde*ϵ - kgrid[0]
                kp_new[z_idx, K_idx, ϵ_idx, :thres_idx] = kgrid[0]
                # non-binding case
                c_tilde_interp = lambda a: interp(k_tilde, c_tilde, a)
                c_new[z_idx, K_idx, ϵ_idx, thres_idx:] = c_tilde_interp(kgrid[thres_idx:])
                kp_new[z_idx, K_idx, ϵ_idx, thres_idx:] = (r+1-δ)*kgrid[thres_idx:] + w*l_tilde*ϵ - c_new[z_idx, K_idx, ϵ_idx, thres_idx:]
        
    return c_new, kp_new

In [7]:
%%time
# Example: EGM_operator

# Give necessaries
φ_b0 = np.mean(Kgrid); φ_b1 = 0
φ_g0 = np.mean(Kgrid); φ_g1 = 0
φ_mat = np.array(((φ_b0, φ_b1), (φ_g0, φ_g1)))
c_old = np.ones((zgridsize, Kgridsize, ϵgridsize, kgridsize))

# Solve
c_new, kp_new = EGM_operator(φ_mat, c_old, zgrid, zgridsize, Kgrid, Kgridsize, ϵgrid, ϵgridsize, kgrid, kgridsize, ugrid, Lgrid, l_tilde, Π, α, δ)

CPU times: user 1.7 ms, sys: 18 µs, total: 1.72 ms
Wall time: 1.74 ms


In [8]:
@njit
def EGM_iterator(φ_mat, c_0, 
                 zgrid, zgridsize, Kgrid, Kgridsize, ϵgrid, ϵgridsize, kgrid, kgridsize, ugrid, Lgrid, l_tilde, Π, α, δ, tol, max_iter):
    # An iterator to solve HH problem by the endogenous grid method for given coefficients
    
    c_old = c_0
    
    for iter in range(max_iter):
        c_new, kp_new = EGM_operator(φ_mat, c_old, zgrid, zgridsize, Kgrid, Kgridsize, ϵgrid, ϵgridsize, kgrid, kgridsize, ugrid, Lgrid, l_tilde, Π, α, δ)
        
        # Updating
        if np.max(np.abs(c_new - c_old)) < tol:
            c_star = c_new
            kp_star = kp_new
            return c_star, kp_star
        else:
            c_old = c_new
        
    # Check convergence
    if iter == max_iter-1:
        print("Error: No convergence in EGM_iterator function")

In [10]:
%%time
# Example: EGM_iterator

# Give necessaries
φ_b0 = np.mean(Kgrid); φ_b1 = 0
φ_g0 = np.mean(Kgrid); φ_g1 = 0
φ_mat = np.array(((φ_b0, φ_b1), (φ_g0, φ_g1)))
c_0 = np.ones((zgridsize, Kgridsize, ϵgridsize, kgridsize))

# Solve
c_star, kp_star = EGM_iterator(φ_mat, c_0, zgrid, zgridsize, Kgrid, Kgridsize, ϵgrid, ϵgridsize, kgrid, kgridsize, ugrid, Lgrid, l_tilde, Π, α, δ, tol, max_iter)

CPU times: user 155 ms, sys: 857 µs, total: 156 ms
Wall time: 157 ms


## 2. Simulation

In [11]:
@njit
def mu_operator(kp_star, mu_old, z_idx, zp_idx, 
                ϵgridsize, kgrid, kgridsize, Π):
    # An operator to update individuals' distribution by the step function approximation method
    
    # Pre-allocation
    mu_new = np.zeros((ϵgridsize, kgridsize))
    
    # Calculate aggregate capital
    K = np.dot(np.sum(mu_old, 0), kgrid)

    for ϵ_idx in range(ϵgridsize):
        for k_idx in range(kgridsize):
            
            # Interpolate the policy function with respect to K            
            kp = interp(Kgrid, kp_star[z_idx, :, ϵ_idx, k_idx], K)
            
            if kp > kgrid[-1]:
                mu_new[:, -1] += mu_old[ϵ_idx, k_idx] * (Π[z_idx, zp_idx, ϵ_idx, :] / np.sum(Π[z_idx, zp_idx, ϵ_idx, :]))
            else:
                idx_r = np.where(kgrid > kp)[0][0]
                idx_l = idx_r - 1
                weight = (kgrid[idx_r]-kp) / (kgrid[idx_r]-kgrid[idx_l])
                mu_new[:, idx_l] += mu_old[ϵ_idx, k_idx] * (Π[z_idx, zp_idx, ϵ_idx, :] / np.sum(Π[z_idx, zp_idx, ϵ_idx, :])) * weight
                mu_new[:, idx_r] += mu_old[ϵ_idx, k_idx] * (Π[z_idx, zp_idx, ϵ_idx, :] / np.sum(Π[z_idx, zp_idx, ϵ_idx, :])) * (1-weight)
                
    return mu_new

In [13]:
%%time
# Example: mu_operator

# Give necessaries
mu_old = np.ones((ϵgridsize, kgridsize)) / (ϵgridsize*kgridsize)
z_idx = 1; zp_idx = 1

# Solve
mu_new = mu_operator(kp_star, mu_old, z_idx, zp_idx, ϵgridsize, kgrid, kgridsize, Π)

CPU times: user 227 µs, sys: 1e+03 ns, total: 228 µs
Wall time: 231 µs


In [14]:
@njit
def Simulate_forward(kp_star, mu_0, 
                     zgrid, ϵgridsize, kgrid, kgridsize, Π, z_series, T, T_0):
    # A simulator which generates mu & K series for given z_series
    
    # Pre-allocation
    mu_series = np.empty((T, ϵgridsize, kgridsize))
    K_series = np.empty(T)
    
    # Initial guess
    mu_series[0, :, :] = mu_0
    K0 = np.dot(np.sum(mu_0, 0), kgrid)
    K_series[0] = K0
    
    for i in range(T-1):
        z_idx = np.where(z_series[i] == zgrid)[0][0]
        zp_idx = np.where(z_series[i+1] == zgrid)[0][0]
        mu_series[i+1, :, :] = mu_operator(kp_star, mu_series[i, :, :], z_idx, zp_idx, ϵgridsize, kgrid, kgridsize, Π)
        K_series[i+1] = np.dot(np.sum(mu_series[i+1, :, :], 0), kgrid)
        
    K_series_pure = K_series[T_0:]
    z_series_pure = z_series[T_0:]
    return K_series_pure, z_series_pure

In [16]:
%%time
# Example: Simulate_forward

# Give necessaries
mu_0 = np.ones((ϵgridsize, kgridsize)) / (ϵgridsize*kgridsize)

# Solve
K_series_pure, z_series_pure = Simulate_forward(kp_star, mu_0, zgrid, ϵgridsize, kgrid, kgridsize, Π, z_series, T, T_0)
K_series_pure # plausible results?

CPU times: user 1.68 s, sys: 5.85 ms, total: 1.69 s
Wall time: 1.7 s


array([1.03868492, 1.09678516, 1.14501251, ..., 1.38565844, 1.38768744,
       1.38940801])

## 3. OLS

In [17]:
#@njit
def Coef_OLS(K_series_pure, z_series_pure, 
             zgrid):
    
    # Divide samples
    if z_series_pure[-1] == zgrid[0]:
        b_idx = np.where(z_series_pure == zgrid[0])[0]
        Kb_idx = np.delete(b_idx, -1)
        K_vec_b = K_series_pure[Kb_idx]
        Kp_vec_b = K_series_pure[Kb_idx + 1]
        K_vec_g = K_series_pure[np.where(z_series_pure == zgrid[1])[0]]
        Kp_vec_g = K_series_pure[np.where(z_series_pure == zgrid[1])[0] + 1]
    else:
        g_idx = np.where(z_series_pure == zgrid[1])[0]
        Kg_idx = np.delete(g_idx, -1)
        K_vec_b = K_series_pure[np.where(z_series_pure == zgrid[0])[0]]
        Kp_vec_b = K_series_pure[np.where(z_series_pure == zgrid[0])[0] + 1]
        K_vec_g = K_series_pure[Kg_idx]
        Kp_vec_g = K_series_pure[Kg_idx + 1]
        
    # Run the regression
    #model = SM.OLS(np.log(Kp_vec_b), np.transpose(np.array([np.ones(np.size(K_vec_b)), np.log(K_vec_b)])))
    logK_vec_b = SM.add_constant(np.log(K_vec_b))
    model = SM.OLS(np.log(Kp_vec_b), logK_vec_b)
    results = model.fit()
    φ_b = results.params
    R_squared_b = results.rsquared
    
    #model = SM.OLS(np.log(Kp_vec_g), np.transpose(np.array([np.ones(np.size(K_vec_g)), np.log(K_vec_g)])))
    logK_vec_g = SM.add_constant(np.log(K_vec_g))
    model = SM.OLS(np.log(Kp_vec_g), logK_vec_g)
    results = model.fit()
    results.params
    φ_g = results.params
    R_squared_g = results.rsquared
    
    φ_mat_new = np.array((φ_b, φ_g))
    R_squared = np.array((R_squared_b, R_squared_g))
    
    return φ_mat_new, R_squared

In [19]:
%%time
# Example: Simulate_forward

# Solve
φ_mat_new, R_squared = Coef_OLS(K_series_pure, z_series_pure, zgrid)
print(φ_mat_new, R_squared) # plausible results?

[[ 0.07221296  0.7515215 ]
 [-0.08472005  0.88707995]] [0.99756345 0.99909395]
CPU times: user 54.9 ms, sys: 7.3 ms, total: 62.2 ms
Wall time: 8.73 ms


## 4. Final: Iterate to find true coefficients

In [20]:
#@njit
def φ_mat_operator(φ_mat, 
                   zgrid, zgridsize, Kgrid, Kgridsize, ϵgrid, ϵgridsize, kgrid, kgridsize, ugrid, Lgrid, l_tilde, Π, z_series, T, T_0, α, δ, tol, max_iter):
    
    # Initial guess
    c_0 = np.ones((zgridsize, Kgridsize, ϵgridsize, kgridsize))
    mu_0 = np.ones((ϵgridsize, kgridsize)) / (ϵgridsize*kgridsize)
    
    # Step 1: HH problem
    c_star, kp_star = EGM_iterator(φ_mat, c_0, zgrid, zgridsize, Kgrid, Kgridsize, ϵgrid, ϵgridsize, kgrid, kgridsize, ugrid, Lgrid, l_tilde, Π, α, δ, tol, max_iter)
    
    # Step 2: Simulate forward
    K_series_pure, z_series_pure = Simulate_forward(kp_star, mu_0, zgrid, ϵgridsize, kgrid, kgridsize, Π, z_series, T, T_0)
    
    # Step 3: Calculate coefficients
    φ_mat_new, R_squared = Coef_OLS(K_series_pure, z_series_pure, zgrid)
    
    return φ_mat_new, R_squared

In [22]:
%%time
# Example: φ_mat_operator

# Give necessaries
φ_b0 = np.mean(Kgrid); φ_b1 = 0
φ_g0 = np.mean(Kgrid); φ_g1 = 0
φ_mat = np.array(((φ_b0, φ_b1), (φ_g0, φ_g1)))

# Solve
φ_mat_new, R_squared = φ_mat_operator(φ_mat, zgrid, zgridsize, Kgrid, Kgridsize, ϵgrid, ϵgridsize, kgrid, kgridsize, ugrid, Lgrid, l_tilde, Π, z_series, T, T_0, α, δ, tol, max_iter)

CPU times: user 1.93 s, sys: 35 ms, total: 1.96 s
Wall time: 1.89 s


In [23]:
#@njit
def φ_mat_iterator(φ_mat_0, 
                   zgrid, zgridsize, Kgrid, Kgridsize, ϵgrid, ϵgridsize, kgrid, kgridsize, ugrid, Lgrid, l_tilde, z_series, T, T_0, Π, α, δ, tol, max_iter, update_weight, procedure=0):
    
    φ_mat = φ_mat_0
    
    for iter in range(max_iter):
        φ_mat_new, R_squared = φ_mat_operator(φ_mat, zgrid, zgridsize, Kgrid, Kgridsize, ϵgrid, ϵgridsize, kgrid, kgridsize, ugrid, Lgrid, l_tilde, Π, z_series, T, T_0, α, δ, tol, max_iter)        
        
        diff = np.max(np.abs(φ_mat_new - φ_mat))
        # Check progress
        if procedure == 1:
            print(φ_mat_new, R_squared, diff, iter)
        
        # Updating
        if diff < tol:
            φ_mat_star = φ_mat_new
            R_squared_star = R_squared
            return φ_mat_star, R_squared_star
        else:
            φ_mat = update_weight*φ_mat_new + (1-update_weight)*φ_mat
            
    # Check convergence
    if iter == max_iter-1:
        print("Error: No convergence in φ_mat_iterator function")

In [25]:
%%time

# Initial guess
φ_b0 = np.mean(Kgrid); φ_b1 = 0
φ_g0 = np.mean(Kgrid); φ_g1 = 0
φ_mat_0 = np.array(((φ_b0, φ_b1), (φ_g0, φ_g1)))

# Solve
φ_mat_star, R_squared_star = φ_mat_iterator(φ_mat_0, zgrid, zgridsize, Kgrid, Kgridsize, ϵgrid, ϵgridsize, kgrid, kgridsize, ugrid, Lgrid, l_tilde, z_series, T, T_0, Π, α, δ, tol, max_iter, update_weight, procedure=1)

[[ 0.07221296  0.7515215 ]
 [-0.08472005  0.88707995]] [0.99756345 0.99909395] 12.584720047911338 0
[[ 0.07247429  0.75217495]
 [-0.08411775  0.88751012]] [0.99757887 0.99910165] 10.067173741927789 1
[[ 0.07313954  0.75384036]
 [-0.08259321  0.88860042]] [0.99761794 0.99912121] 8.052214452298005 2
[[ 0.07447632  0.75725103]
 [-0.07952347  0.89081942]] [0.99769623 0.99916086] 6.438701821585466 3
[[ 0.07666341  0.76306601]
 [-0.0744222   0.8945818 ]] [0.9978283  0.99922756] 5.1458601833391775 4
[[ 0.07965696  0.77169906]
 [-0.06716597  0.90000674]] [0.99802148 0.99931847] 4.109431925405556 5
[[ 0.08312457  0.78322359]
 [-0.05810922  0.90665041]] [0.99826725 0.9994115 ] 3.27848878919279 6
[[ 0.08656172  0.79705547]
 [-0.04809091  0.91467705]] [0.9985391  0.99952173] 2.612772713626305 7
[[ 0.08937875  0.8124782 ]
 [-0.03799772  0.92321472]] [0.99881223 0.99962503] 2.0801249833956437 8
[[ 0.09014707  0.83151484]
 [-0.02622564  0.93153877]] [0.99915209 0.999736  ] 1.6585555969533503 9
[[ 0.0

In [26]:
φ_mat_star, R_squared_star

(array([[0.08794625, 0.96522389],
        [0.09459945, 0.96418105]]),
 array([0.99999932, 0.99999948]))